In [1]:
from tensorflow.keras.datasets import cifar10
import numpy as np

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
#X_train = np.expand_dims(X_train, 3)
#X_test = np.expand_dims(X_test, 3)



In [2]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 1)
(10000, 1)


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=X_train[0].shape))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

In [4]:
model.compile(optimizer="Adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=32, epochs=15, verbose=2, validation_data=(X_test, y_test))

Epoch 1/15
1563/1563 - 17s - loss: 2.5886 - accuracy: 0.2191 - val_loss: 1.9559 - val_accuracy: 0.3090 - 17s/epoch - 11ms/step
Epoch 2/15
1563/1563 - 10s - loss: 1.8127 - accuracy: 0.3572 - val_loss: 1.6667 - val_accuracy: 0.3999 - 10s/epoch - 6ms/step
Epoch 3/15
1563/1563 - 9s - loss: 1.5812 - accuracy: 0.4335 - val_loss: 1.5590 - val_accuracy: 0.4332 - 9s/epoch - 6ms/step
Epoch 4/15
1563/1563 - 8s - loss: 1.4763 - accuracy: 0.4765 - val_loss: 1.4544 - val_accuracy: 0.4780 - 8s/epoch - 5ms/step
Epoch 5/15
1563/1563 - 8s - loss: 1.3808 - accuracy: 0.5145 - val_loss: 1.3652 - val_accuracy: 0.5135 - 8s/epoch - 5ms/step
Epoch 6/15
1563/1563 - 11s - loss: 1.3076 - accuracy: 0.5429 - val_loss: 1.3225 - val_accuracy: 0.5388 - 11s/epoch - 7ms/step
Epoch 7/15
1563/1563 - 9s - loss: 1.2574 - accuracy: 0.5600 - val_loss: 1.2743 - val_accuracy: 0.5604 - 9s/epoch - 6ms/step
Epoch 8/15
1563/1563 - 9s - loss: 1.2214 - accuracy: 0.5767 - val_loss: 1.2282 - val_accuracy: 0.5768 - 9s/epoch - 6ms/step
E

In [ ]:
import matplotlib.pyplot as plt

figures, axes = plt.subplots(1,2,figsize=(10,5))

axes[0].plot(history.history['accuracy'])
axes[0].plot(history.history['val_accuracy'])
axes[0].set_title('Model Accuracy vs. Iterations')
axes[0].set_ylabel('Accuracy')
axes[0].set_xlabel('Iterations')
axes[0].legend(['Train', 'Validation'], loc='lower right')

axes[1].plot(history.history['loss'])
axes[1].plot(history.history['val_loss'])
axes[1].set_title('Model Loss vs. Iterations')
axes[1].set_ylabel('Loss')
axes[1].set_xlabel('Iterations')
axes[1].legend(['Train', 'Validation'], loc='upper right')

plt.show()

: 

: 